### tf.data and optimizer

TF 的基础笔记 2，灵感来源于 [Stanford CS20si 课程](http://web.stanford.edu/class/cs20si/syllabus.html)，enjoy！

#### TF.data
- 使用 placeholder 和 feed_dict 的好处是数据的控制是在 tf 之外的，很 pythonic，并且容易 shuffle, batch, random。但是不好的地方时，数据处理是单线程的，降低速度。

In [ ]:
data, n_samples = utils.read_birth_life_data(DATA_FILE)
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')
with tf.Session() as sess:
       ...
    # Step 8: train the model
    for i in range(100): # run 100 epochs
        for x, y in data:
            # Session runs train_op to minimize loss
            sess.run(optimizer, feed_dict={X: x, Y:y}) 

我们可以将数据存在 tf.data.Dataset 对象中

In [ ]:
# 1. 初始化
# features 和 labels 都是 tensor
tf.data.Dataset.from_tensor_slices((features, labels))
# 因为 tf 和 np 是无缝集成的，因此也可以传入 np array，所以初始化可以变为：
dataset = tf.data.Dataset.from_tensor_slices((data[:,0], data[:,1]))
# sanity check
print(dataset.output_types) # >> (tf.float32, tf.float32)
print(dataset.output_shapes) # >> (TensorShape([]), TensorShape([]))
# 也可以直接从文件中创建 Dataset
tf.data.TextLineDataset(filenames)
tf.data.FixedLengthRecordDataset(filenames)
tf.data.TFRecordDataset(filenames)
# 2. 创建迭代器
# 创建迭代一次的迭代器
iterator = dataset.make_one_shot_iterator()
X, Y = iterator.get_next()
# 使用方法
with tf.Session() as sess:
    print(sess.run([X, Y])) # >> [1.822, 74.82825]
    print(sess.run([X, Y])) # >> [3.869, 70.81949]
    print(sess.run([X, Y])) # >> [3.911, 72.15066]
# 可以直接调用 optimizer 来执行运算，再也不需要 feed_dict 了
# 注意 TF 并没有捕捉 OutOfRangeError 异常，因此我们需要自己捕捉
for i in range(100): # train the model 100 epochs
        total_loss = 0
        try:
            while True:
                sess.run([optimizer]) 
        except tf.errors.OutOfRangeError:
            pass
# 创建可以反复迭代的迭代器
iterator = dataset.make_initializable_iterator()
...
for i in range(100): 
        sess.run(iterator.initializer) 
        total_loss = 0
        try:
            while True:
                sess.run([optimizer]) 
        except tf.errors.OutOfRangeError:
            pass
# 3. 设置 batch，shuffle，repeat 甚至是数据转换
dataset = dataset.shuffle(1000)
dataset = dataset.repeat(100)
dataset = dataset.batch(128)
dataset = dataset.map(lambda x: tf.one_hot(x, 10)) # convert each element of dataset to one_hot vector

# 数据比较
# 在 Macbook Pro with 2.7 GHz Intel Core i5
# with placeholder took on average 9.05271519 seconds
# with tf.data took on average 6.12285947 seconds
# 32.4% faster

#### TF Optimizer

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
sess.run([optimizer])

optimizer 是一个 operator，它会在图中寻找 loss 依赖的节点，并且对可以训练的节点做自动偏微分。

In [ ]:
# Variable 中的 trainerable 参数控制是否需要被更新。
# 一个例子是 global_step 变量，就需要将它的 trainerable 设置为 false
global_step = tf.Variable(0, trainable=False, dtype=tf.int32)
learning_rate = 0.01 * 0.99 ** tf.cast(global_step, tf.float32)
increment_step = global_step.assign_add(1)
optimizer = tf.train.GradientDescentOptimizer(learning_rate) # learning rate can be a tensor

可以设置让 optimizer 只计算指定的梯度。在计算的过程中，你还可以更改算出的梯度，并反馈回去。

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
# compute the gradients for a list of variables.
grads_and_vars = optimizer.compute_gradients(loss, <list of variables>)
# grads_and_vars is a list of tuples (gradient, variable).  Do whatever you
# need to the 'gradient' part, for example, subtract each of them by 1.
subtracted_grads_and_vars = [(gv[0] - 1.0, gv[1]) for gv in grads_and_vars]
# ask the optimizer to apply the subtracted gradients.
optimizer.apply_gradients(subtracted_grads_and_vars)

optimizer 会自动计算图中所有变量的导数。但是你也可以让 tf 去计算某些特别变量的导数，使用 tf.gradients。
- 这个方法会计算 ys 对 xs 的偏导数，是一个 tensor 或者 tensor 的列表
- grad_ys 是用来记录 ys 的梯度的，和 ys 形状相同

当只训练模型的一部分时，这个方法很有用，例如使用 tf.gradients() 来计算 loss 相对于中间层网络的梯度 G。然后我们只需要优化中间层的输出 M 和 M+G 的差别最小即可。这只需要更新神经网络的前半部分。（差别最小，即收敛。最优化的目标就是梯度最小化。）

In [ ]:
tf.gradients(
    ys,
    xs,
    grad_ys=None,
    name='gradients',
    colocate_gradients_with_ops=False,
    gate_gradients=False,
    aggregation_method=None,
    stop_gradients=None
)

下面罗列出一系列的优化函数。目前来看，Adam 优化是最好的选择。

In [ ]:
tf.train.Optimizer
tf.train.GradientDescentOptimizer
tf.train.AdadeltaOptimizer
tf.train.AdagradOptimizer
tf.train.AdagradDAOptimizer
tf.train.MomentumOptimizer
tf.train.AdamOptimizer
tf.train.FtrlOptimizer
tf.train.ProximalGradientDescentOptimizer
tf.train.ProximalAdagradOptimizer
tf.train.RMSPropOptimizer